<a href="https://colab.research.google.com/github/nupursjsu/Deep-Learning/blob/master/Graded_Assignment4/PyTorch_LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing LeNet using PyTorch (Using MNIST dataset)

## Importing required libraries

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from PIL import Image
import torch.nn.functional as F

## Loading MNIST dataset

In [0]:
def load_data(train_batch_size, test_batch_size):
    # Fetch training data: total 60000 samples
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Resize((32, 32)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=train_batch_size, shuffle=True)

    # Fetch test data: total 10000 samples
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=test_batch_size, shuffle=True)

    return (train_loader, test_loader)

## Defining LeNet model

In [0]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Defining the train function

In [0]:
def train(model, optimizer, epoch, train_loader, log_interval):  
    model.train()

    #Defining loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    #Iterating over batches of data
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = Variable(data), Variable(target)

        #Setting the gradients to zero, since PyTorch accumulates them
        optimizer.zero_grad()

        #Defining Forward propagation
        output = model(data)

        loss = loss_fn(output, target)

        #Defining Backward propagation
        loss.backward()

        #Updating the parameters after backward prop
        optimizer.step()

        #Displaying the log
        if batch_idx % log_interval == 0:
            print('Train set, Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                loss.item()))

##Defining the test function

In [0]:
def test(model, epoch, test_loader):
    #Defining that we are testing the model
    model.eval()

    #Initializing the variables for init loss & correct prediction accumulators
    test_loss = 0
    correct = 0

    #Defining loss function
    loss_fn = torch.nn.CrossEntropyLoss(size_average=False)

    #Iterating over data
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        
        #Defining Forward propagation
        output = model(data)

        #Calculating & accumulating loss
        test_loss += loss_fn(output, target).item()

        #Getting the index of the max log-probability which is the predicted output label
        pred = np.argmax(output.data, axis=1)

        #If correct, increment correct prediction accumulator
        correct = correct + np.equal(pred, target.data).sum()

    #Displaying the log
    test_loss /= len(test_loader.dataset)
    print('\nTest set, Epoch {} , Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

##Defining the parameters

In [0]:
torch.manual_seed(123)

model = LeNet()

lr = 0.01
momentum=0.5
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

train_batch_size = 64
test_batch_size = 1000
train_loader, test_loader = load_data(train_batch_size, test_batch_size)

##Training and Testing the LeNet model

In [55]:
epochs = 10
log_interval = 100
for epoch in range(1, epochs + 1):
    train(model, optimizer, epoch, train_loader, log_interval=log_interval)
    test(model, epoch, test_loader)

Train set, Epoch 1 [0/60000 (0%)]	Loss: 2.297229
Train set, Epoch 1 [6400/60000 (11%)]	Loss: 2.138053
Train set, Epoch 1 [12800/60000 (21%)]	Loss: 0.612845
Train set, Epoch 1 [19200/60000 (32%)]	Loss: 0.288354
Train set, Epoch 1 [25600/60000 (43%)]	Loss: 0.421566
Train set, Epoch 1 [32000/60000 (53%)]	Loss: 0.266445
Train set, Epoch 1 [38400/60000 (64%)]	Loss: 0.257292
Train set, Epoch 1 [44800/60000 (75%)]	Loss: 0.118953
Train set, Epoch 1 [51200/60000 (85%)]	Loss: 0.157034
Train set, Epoch 1 [57600/60000 (96%)]	Loss: 0.124597


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set, Epoch 1 , Average loss: 0.1271, Accuracy: 9595/10000 (96%)

Train set, Epoch 2 [0/60000 (0%)]	Loss: 0.122956
Train set, Epoch 2 [6400/60000 (11%)]	Loss: 0.056236
Train set, Epoch 2 [12800/60000 (21%)]	Loss: 0.260173
Train set, Epoch 2 [19200/60000 (32%)]	Loss: 0.111177
Train set, Epoch 2 [25600/60000 (43%)]	Loss: 0.087242
Train set, Epoch 2 [32000/60000 (53%)]	Loss: 0.079405
Train set, Epoch 2 [38400/60000 (64%)]	Loss: 0.154342
Train set, Epoch 2 [44800/60000 (75%)]	Loss: 0.043412
Train set, Epoch 2 [51200/60000 (85%)]	Loss: 0.111378
Train set, Epoch 2 [57600/60000 (96%)]	Loss: 0.094715

Test set, Epoch 2 , Average loss: 0.1065, Accuracy: 9654/10000 (97%)

Train set, Epoch 3 [0/60000 (0%)]	Loss: 0.141730
Train set, Epoch 3 [6400/60000 (11%)]	Loss: 0.028807
Train set, Epoch 3 [12800/60000 (21%)]	Loss: 0.060322
Train set, Epoch 3 [19200/60000 (32%)]	Loss: 0.055607
Train set, Epoch 3 [25600/60000 (43%)]	Loss: 0.076396
Train set, Epoch 3 [32000/60000 (53%)]	Loss: 0.041063
Train 

##Conclusion

We could see LeNet model is giving an accuracy of 99% on MNIST dataset which is pretty good.